In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/stability_paper_data/Perovskite_Stability_with_features.csv')

In [4]:
df.dtypes

Material Composition       object
A site #1                  object
A site #2                  object
A site #3                  object
B site #1                  object
                            ...  
host_Bsite0_At. #           int64
host_Bsite0_IsHexagonal     int64
host_Bsite0_IsNoblegas      int64
Asite_IsAlkali_max          int64
Bsite_IsMetal_max           int64
Length: 81, dtype: object